In [25]:
import pickle as pkl
comment_input_file = '/Users/allen/Codes/NLP_learning/Comment_extraction/dzdp_hot_SH.txt'
# comment_input_file = '/Users/allen/Codes/NLP_learning/Web_crawler/cmb.txt'

comments = []
with open(comment_input_file, 'r') as f:
    for line in f:
        if line != '\n': comments.append(line)

len(comments)

8775

In [26]:
import re
comments_dp = []
import jieba.posseg as pseg

for comment in comments:
    comment_to_list = re.split(' |,|，|。|;|\~|…|\?|？', comment)
    comment_cut_list = [pseg.cut(c) for c in comment_to_list]
    for comment_cut in comment_cut_list:
        comments_dp.append([(w, v) for w,v in comment_cut])

In [54]:
# load LTP
from pyltp import Parser
parser = Parser()
parser.load('/Users/allen/Codes/pyltp/ltp_data_v3.4.0/parser.model')

# use LTP to postag
from pyltp import Postagger
postagger = Postagger()
postagger.load('/Users/allen/Codes/pyltp/ltp_data_v3.4.0/pos.model')

In [55]:

# test_seqs = [t[0] for t in comments_dp[10]]
test_seqs = '小米,手机,外观,看,起来,还是,很,大气,的'.split(',')
postags = postagger.postag(test_seqs)
print(' '.join(postags))

n n n v v d d a u


In [56]:
test_seqs

['小米', '手机', '外观', '看', '起来', '还是', '很', '大气', '的']

In [57]:
# parser
arcs = parser.parse(test_seqs, postags)
rely_ids = [arc.head for arc in arcs]
relations = [arc.relation for arc in arcs]
heads = ['Root' if _id == 0 else test_seqs[_id-1] for _id in rely_ids]
for i in range(len(test_seqs)):
    print(relations[i] + '(' + test_seqs[i] + ',' + heads[i] + ')')

ATT(小米,手机)
ATT(手机,外观)
SBV(外观,看)
HED(看,Root)
CMP(起来,看)
CMP(还是,看)
ADV(很,大气)
VOB(大气,还是)
RAD(的,大气)


In [58]:
from pyltp import Segmentor
segmentor = Segmentor()
segmentor.load('/Users/allen/Codes/pyltp/ltp_data_v3.4.0/cws.model')

In [66]:
comments_parsed = []
import pdb
for comment in comments_dp:
    seq = [w[0] for w in comment]
    seq = ''.join(seq)
    seq = segmentor.segment(seq)
    if len(seq) > 20:
        continue
    postags = postagger.postag(seq)
    arcs = parser.parse(seq,postags)
    rely_ids = [arc.head for arc in arcs]
    relations = [arc.relation for arc in arcs]
    heads = ['Root' if _id == 0 else seq[_id-1] for _id in rely_ids]
    if not relations:
        continue
#     print(seq)
#     for i in range(len(test_seqs)):
#         print(relations[i] + '(' + seq[i] + ',' + heads[i] + ')')
    with open('parser_result.txt', 'a') as f:
        res = []
        for i in range(len(seq)):
            res.append(relations[i] + '(' + seq[i] + ',' + heads[i] + ')')
        f.write(','.join(seq) + '\n')
        f.write('\t'.join(res) + '\n')
        f.write('\n')
    for i in range(len(seq)):
        if relations[i] == 'SBV':
            n, v = seq[i], heads[i]
            d = ''
            for j in range(len(seq)):
                if relations[j] in ['ADV','CMP'] and heads[j] == v:
                    d = seq[j]
            res = []
            if not d:
                res = [n,v]
            else:
                res = [n,d,v]
            comments_parsed.append(res)
            

In [75]:
parser.release()
segmentor.release()
postagger.release()

In [11]:
word_vect_file = '/Users/allen/Codes/NLP_learning/Word_emb/bdbk_dict.pkl'
with open(word_vect_file, 'rb') as f:
    word_vect_dict = pkl.load(f)

In [68]:
comments_vectors = []
comments_filtered = []
for comment in comments_parsed:
    vec = [0 for _ in range(300)]
    n = 0
    for w in comment:
        if w in word_vect_dict:
            v = word_vect_dict[w]
            for i, val in enumerate(v):
                vec[i] += v[i]
            n += 1
    if n:
        comments_vectors.append([v/n for v in vec])
        comments_filtered.append(comment)

In [69]:
len(comments_vectors), len(comments_filtered), len(comments_parsed)

(92590, 92590, 92973)

In [70]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
n_clusters = 30
clusterer = KMeans(n_clusters=30, random_state=10)
# clusterer = DBSCAN(eps=0.05)
cluster_labels = clusterer.fit_predict(comments_vectors)

In [71]:
labels = set()
for l in cluster_labels:
    labels.add(l)
n_clusters = len(labels)
n_clusters

30

In [44]:
cluster_labels

array([-1, -1, -1, ..., -1, -1, -1])

In [72]:
with open('Parser_cluster_result.txt', 'w') as f:
    for label_idx in range(n_clusters):
        idx = [i for i,l in enumerate(cluster_labels.tolist()) if l == label_idx]
        f.write('Cluster: ' + str(label_idx) + '---------------\n')
        for _i in idx:
            f.write(','.join(comments_filtered[_i]) + '\n')

In [ ]:
with open('comments_parsed.txt', 'w') as f:
    for c in comments_parsed:
        f.write(','.join(c))

In [78]:
stop_words = set()
with open('stop_words.txt', 'r') as f:
    for line in f:
        w = line.strip()
        stop_words.add(w)


In [79]:
def combine(n_s, d_s, a_s, idx):
    if not n_s or not a_s:
        return []
    if len(n_s) > 5:
        return []
    res = []
    for n in n_s:
        for a in a_s:
            if not d_s:
                res.append([n, a])
            else:
                for d in d_s:
                    res.append([n, d, a])
    return res
comments_extracted = []
for idx,comment in enumerate(comments_dp):
    _n = [w for w, v in comment if v == 'n']
    _d = [w for w, v in comment if v == 'd' and w not in stop_words]
    _a = [w for w, v in comment if v == 'a']
    comments_extracted += combine(_n, _d, _a, idx)
with open('comments_extracted.txt', 'w') as f:
    for comment in comments_extracted:
        f.write(' '.join(comment) + '\n')

In [80]:
import gensim
dictionary = gensim.corpora.Dictionary(comments_extracted)
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in comments_extracted]


In [81]:
lda_model =  gensim.models.LdaModel(bow_corpus, 
                                   num_topics = 100, 
                                   id2word = dictionary,                                    
                                   passes = 10)

In [82]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {}  \nWords: {}".format(idx,  topic ))
    print("\n")

Topic: 0  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 1  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 2  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 3  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 4  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 5  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.001*"热" + 0.001*"网银"


Topic: 6  
Words: 0.001*"相继" + 0.001*"双方" + 0.001*"不错" + 0.001*"状态" + 0.001*"跑步" + 0.001*"账单" + 0.001*"老" + 0.001*"过硬" + 0.0

In [65]:
import numpy as np

comments_scores = []
for comment in comments_extracted:
    bow_vector = dictionary.doc2bow(comment)    
    comments_scores.append(lda_model.get_document_topics(bow_vector))
comments_processed = np.zeros((len(comments_scores), 20))
for idx, test_vector in enumerate(comments_scores):
    for t in test_vector:
        comments_processed[idx][t[0]] = t[1]

In [69]:
nc = 100
from sklearn.cluster import KMeans
clu = KMeans(n_clusters=nc, random_state=10)
clu_labels = clu.fit_predict(comments_processed.tolist())
clu_distance = clu.transform(comments_processed.tolist()).tolist()
clu_dist = []
for c_l in clu_distance:
    clu_dist.append(sum([c**2 for c in c_l]))

In [72]:
clu_dist

[32.11261821409896,
 36.887937629071985,
 31.841722728316185,
 36.887935845983954,
 36.887935845983954,
 36.343606085504554,
 32.78459303570827,
 33.32487078529578,
 31.433417823819436,
 44.4311536154349,
 33.339069821105134,
 31.501017432357337,
 31.795822819665922,
 37.082914081601324,
 34.632329802521646,
 35.02538650803577,
 32.960218151168746,
 31.12228490943819,
 31.41707382951914,
 32.880262834953456,
 31.042324930903057,
 31.337109295896994,
 31.273696940716107,
 32.165926389082394,
 31.273696940716107,
 37.90105673788667,
 37.667371423128095,
 37.57053658738256,
 59.559459677297404,
 36.00185553822268,
 35.76808370738349,
 41.27632012182586,
 40.925172570407966,
 37.66819462710981,
 60.21980632018,
 38.62043962072825,
 38.54910923693112,
 36.55542007051747,
 41.56490228445152,
 36.627148164350366,
 37.75611388152228,
 46.49482319969893,
 46.28979238690627,
 46.49482294225293,
 46.28978686757717,
 33.4707184838851,
 45.76552931705121,
 33.47071603928488,
 45.76553065059938,
 37

In [75]:
with open('LDA_Kmeans_result.txt', 'w') as f:
    for i in range(nc):
        f.write(str(i) + '---------------------------------------' + '\n')
        comments_to_wr = [s for idx, (s, l) in enumerate(zip(comments_extracted,clu_labels)) if l == i]
        idx_to_wr = [idx for idx, (s, l) in enumerate(zip(comments_extracted,clu_labels)) if l == i]
        
        ranked_comments = list(zip(idx_to_wr, comments_to_wr))
        ranked_comments.sort()
        for s, c in ranked_comments[:10]:
            f.write(' '.join(c) + ' ' + str(s) + '\n')
#         for s,idx in zip(comments_to_wr, idx_to_wr):
#             f.write(' '.join(s) + ' ' + str(clu_dist[idx]) + '\n')

In [74]:
del word_vect_dict

NameError: name 'word_vect_dict' is not defined